## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [1]:
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

from cp_omero_toolbox import *

import ezomero

import cellprofiler_core.preferences as cp_preferences

import pandas as pd
import tempfile


Bad key text.latex.preview in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95    

In [2]:
# Make CellProfiler run without a GUI
cp_preferences.set_headless()

# Tell CellProfiler to get input from and save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
input_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)
cp_preferences.set_default_image_directory(input_dir.name)

Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [15]:
# Creating a connection object
host = "bioimage.france-bioinformatique.fr"
port = 4074
conn = ezomero.connect(host=host, port=port)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that we can tell our new connection to say "Hi, I'm still here"
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

True

Time to grasp a Dataset from OMERO and download a CellProfiler pipeline that is attached to it. Go to the browser, select a dataset and copy the ID.

In [16]:
dataset_id = int(input("Dataset id: "))


1
Downloaded /tmp/tmp0vrc31s_/Megane_SpotInNuclei-encours_test_V3.cppipe


We create a new pipeline with that file, and we remove the first 4 modules. The first 4 modules are in charge of preparing the image data when they are loaded from disk. We don't need them here because we are using OMERO.


In [ ]:
measurements_dfs = run_cp_pipeline(cp_pipeline_path=cp_pipeline_path,
                                   conn=conn,
                                   dataset_id=dataset_id,
                                   objects_to_image_table="Spots",
                                   objects_to_mask="Nuclei",
                                   objects_to_point="Spots",
                                   output_dir=output_dir)

We can now start feeding images into the pipeline

In [17]:
cols = measurements_dfs["Image"].columns
vals = [measurements_dfs["Image"][c].values.tolist() for c in cols]

print(len(vals))
print(vals)



48
[[12.0, 25.0, 25.0, 24.0, 19.0], [238.0, 429.0, 521.0, 365.0, 603.0], [270.0, 694.0, 906.0, 738.0, 1204.0], [0.0, 0.0, 0.0, 0.0, 0.010000000000005116], [0.0, 0.010000000000005116, 0.0, 0.010000000000019327, 0.009999999999990905], [0.46999999999999886, 1.5899999999999892, 1.3900000000000006, 0.47999999999998977, 0.5000000000000142], [0.45999999999999375, 0.700000000000017, 0.7599999999999909, 0.4700000000000131, 0.4899999999999807], [0.04000000000000625, 0.03999999999999204, 0.03999999999999204, 0.03999999999999204, 0.05000000000001137], [3.4200000000000017, 3.519999999999996, 3.489999999999995, 3.5600000000000023, 3.5799999999999983], [2.170000000000016, 2.219999999999999, 2.2600000000000193, 2.239999999999995, 2.3000000000000114], [0.37999999999998124, 0.4099999999999966, 0.3999999999999915, 0.4099999999999966, 0.45999999999999375], [0.23000000000000398, 0.23000000000000398, 0.2400000000000091, 0.23000000000000398, 0.2599999999999909], [0.060000000000002274, 0.07000000000000739, 0.

In [10]:
# remove the output directory
output_dir.cleanup()

# and close the connection to the OMERO server
conn.close()